In [ ]:
#复习：读写文件后要正确关闭文件。
try:
    f = open('/path/to/file', 'r')
    f.read()
finally:
    if f:
        f.close()
        
with open('/path/to/file', 'r') as f:
    f.read()

with
---
并不是只有open()函数返回的fp对象才能使用with语句。实际上，任何对象，只要正确实现了上下文管理，就可以用于with语句。

实现上下文管理是通过\__enter\__和\__exit\__这两个方法实现的。

In [1]:
class Query(object):

    def __init__(self, name):
        self.name = name

    def __enter__(self):
        print('Begin')
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type:
            print('Error')
        else:
            print('End')

    def query(self):
        print('Query info about %s...' % self.name)
        
with Query('Bob') as q:
    q.query()

Begin
Query info about Bob...
End


@contextmanager
---
@contextmanager这个decorator接受一个generator，用yield语句把with ... as var把变量输出出去，然后，with语句就可以正常地工作了。

编写\__enter\__和\__exit\__仍然很繁琐，因此Python的标准库contextlib提供了更简单的写法，上面的代码可以改写如下：

In [2]:
from contextlib import contextmanager

class Query(object):

    def __init__(self, name):
        self.name = name

    def query(self):
        print('Query info about %s...' % self.name)

@contextmanager
def create_query(name):
    print('Begin')
    q = Query(name)
    yield q
    print('End')


with create_query('Bob') as q:
    q.query()

Begin
Query info about Bob...
End


在某段代码执行前后自动执行特定代码，也可以用@contextmanager实现。

代码的执行顺序是：
1. with语句首先执行yield之前的语句，因此打印出&lt;h1>；
2. yield调用会执行with语句内部的所有语句，因此打印出hello和world；
3. 最后执行yield之后的语句，打印出&lt;/h1>。 

因此，@contextmanager让我们通过编写generator来简化上下文管理。

In [3]:
from contextlib import contextmanager

@contextmanager
def tag(name):
    print("<%s>" % name)
    yield
    print("</%s>" % name)

with tag("h1"):
    print("hello")
    print("world")

<h1>
hello
world
</h1>


@closing
---
如果一个对象没有实现上下文，我们就不能把它用于with语句。这个时候，可以用closing()来把该对象变为上下文对象。例如，用with语句使用urlopen()：

In [5]:
from contextlib import closing
from urllib.request import urlopen

with closing(urlopen('https://www.baidu.com')) as page:
    for line in page:
        print(line)

b'<html>\r\n'
b'<head>\r\n'
b'\t<script>\r\n'
b'\t\tlocation.replace(location.href.replace("https://","http://"));\r\n'
b'\t</script>\r\n'
b'</head>\r\n'
b'<body>\r\n'
b'\t<noscript><meta http-equiv="refresh" content="0;url=http://www.baidu.com/"></noscript>\r\n'
b'</body>\r\n'
b'</html>'


closing也是一个经过@contextmanager装饰的generator，这个generator编写起来其实非常简单。

In [ ]:
@contextmanager
def closing(thing):
    try:
        yield thing
    finally:
        thing.close()

# 把任意对象变为上下文对象，并支持with语句。